In [7]:
#Desarrollamos las funciones que nos piden
import pandas as pd
steam_games = pd.read_parquet(r'C:\\Users\\licle\\Desktop\\Prep Henry\\Proyecto Integrados n1\\PI MLOps - STEAM\\Datasets\\steam_games.parquet')
items_def = pd.read_parquet(r'C:\\Users\\licle\\Desktop\\Prep Henry\\Proyecto Integrados n1\\PI MLOps - STEAM\\Datasets\\users_items.parquet')
reviews_def = pd.read_parquet(r'C:\\Users\\licle\\Desktop\\Prep Henry\\Proyecto Integrados n1\\PI MLOps - STEAM\\Datasets\\user_reviews.parquet')


In [15]:
def PlayTimeGenre(genero : str):
    # Fusionar los DataFrames en función de la columna 'item_id'
    merged_data = pd.merge(steam_games, items_def, left_on='id', right_on='item_id', how='inner')

    # Desanidar las listas de géneros
    genres = merged_data.explode('genres') 

    # Reemplazar "&amp;" por "&" en los géneros
    genres['genres'] = genres['genres'].str.replace('&amp;', 'and')

    # Normalizar los géneros en minúsculas y sin caracteres especiales
    genres['genres'] = genres['genres'].str.lower().str.replace(r'[^a-zA-Z0-9\s]', '').str.strip()

    # Obtener la lista de géneros únicos
    unique_genres = genres['genres'].unique()

    #consulta
    user_genre = genero

    # Normalizar el género ingresado por el usuario
    user_genre = user_genre.lower().strip()

    # Verificar si el género ingresado por el usuario está en la lista de géneros
    if user_genre.lower() not in unique_genres:
        print("Error: El género no está en la lista")
    else:
        # Filtrar los datos según el género ingresado por el usuario
        genre_data = genres[genres['genres'] == user_genre]

    # Agrupar por año y sumar la cantidad de minutos jugados para el género ingresado por el usuario
    minutes_played_by_year = genre_data.groupby(genre_data['release_date'].dt.year)['playtime_forever'].sum()

    # Encontrar el género más jugado según el año más jugado para cada género
    most_played_year = int(minutes_played_by_year.idxmax())
    # Mostrar el año con el género más jugado para el género ingresado por el usuario
    return print("Año de lanzamiento con más horas jugadas para el género", user_genre, ":", most_played_year)

In [39]:
def UserForGenre(genero : str):
    #Fusionar los DataFrames en función de la columna 'item_id'
    merged_data = pd.merge(steam_games, items_def, left_on='id', right_on='item_id', how='inner')

    # Desanidar las listas de géneros
    genres = merged_data.explode('genres')

    # Reemplazar "&amp;" por "&" en los géneros
    genres['genres'] = genres['genres'].str.replace('&amp;', '&')

    # Normalizar los géneros en minúsculas y sin caracteres especiales
    genres['genres'] = genres['genres'].str.lower().str.replace(r'[^a-zA-Z0-9\s]', '').str.strip()
    
    # Obtener la lista de géneros únicos
    unique_genres = genres['genres'].unique()
    
    # Consulta
    user_genre = genero

    # Normalizar el género ingresado por el usuario
    user_genre = user_genre.lower().strip()
    # Verificar si el género ingresado por el usuario está en la lista de géneros
    if user_genre.lower() not in unique_genres:
        print("Error: El género no está en la lista")
    else:
        # Filtrar los datos según el género ingresado por el usuario
        genre_data = genres[genres['genres'] == user_genre]
        # Encontrar el usuario con más minutos jugados para el género especificado
        most_played_user = genre_data.groupby('user_id')['playtime_forever'].sum().idxmax()
        # Filtrar los datos para el usuario con más minutos jugados
        most_played_user_data = genre_data[genre_data['user_id'] == most_played_user]
        # Agrupar los datos por año y sumar los minutos jugados para cada año
        minutes_played_by_year = most_played_user_data.groupby(most_played_user_data['release_date'].dt.year)['playtime_forever'].sum()
        hours_by_year_list = [{"Año": int(year), "Horas": hours/60} for year, hours in minutes_played_by_year.items()]

    # Mostrar la suma de minutos jugados por año para el usuario con más minutos jugados en el género especificado
    
    return print("Usuario con más horas jugadas para Género", user_genre.capitalize(), ":", most_played_user, ' Horas jugadas : ', hours_by_year_list)

In [40]:
UserForGenre('action')

Usuario con más horas jugadas para Género Action : REBAS_AS_F-T  Horas jugadas :  [{'Año': 1998, 'Horas': 0.03333333333333333}, {'Año': 1999, 'Horas': 3.75}, {'Año': 2001, 'Horas': 0.18333333333333332}, {'Año': 2002, 'Horas': 0.016666666666666666}, {'Año': 2003, 'Horas': 31.05}, {'Año': 2004, 'Horas': 35.25}, {'Año': 2005, 'Horas': 54.88333333333333}, {'Año': 2006, 'Horas': 2.3}, {'Año': 2007, 'Horas': 11.4}, {'Año': 2008, 'Horas': 63.18333333333333}, {'Año': 2009, 'Horas': 443.9166666666667}, {'Año': 2010, 'Horas': 235.88333333333333}, {'Año': 2011, 'Horas': 1082.5166666666667}, {'Año': 2012, 'Horas': 1350.8}, {'Año': 2013, 'Horas': 2411.8}, {'Año': 2014, 'Horas': 3574.366666666667}, {'Año': 2015, 'Horas': 3225.516666666667}, {'Año': 2016, 'Horas': 3338.733333333333}, {'Año': 2017, 'Horas': 61.93333333333333}]


In [70]:
def UsersRecommend(año: int):
    reviews_def['item_id'] = reviews_def['item_id'].astype(int)
    steam_games['id'] = steam_games['id'].astype(int)
    
    # Combinar los DataFrames en función del campo 'item_id'
    merged_data = pd.merge(reviews_def, steam_games, left_on='item_id', right_on='id', how='inner')

    # Filtrar los juegos del año dado
    year_games = merged_data[merged_data['release_date'].dt.year == año]

    # Filtrar los juegos recomendados con comentarios positivos o neutros
    recommended_games = year_games[(year_games['recommend'] == True) & (year_games['sentiment_analysis'] >= 1)]

    # Contar el número de recomendaciones para cada juego
    recommend_count = recommended_games['app_name'].value_counts()

    # Obtener el top 3 de juegos más recomendados
    recomendados = recommend_count.head(3)

    # Formatear el resultado según el formato deseado
    respuesta3 = [{"Puesto " + str(i+1): juego} for i, juego in enumerate(recomendados.index)]
    
    return respuesta3

In [71]:
UsersRecommend(2015)

[{'Puesto 1': 'grand theft auto v'},
 {'Puesto 2': 'rocket league®'},
 {'Puesto 3': 'undertale'}]

In [68]:
def UsersNotRecommend(año : int):
    reviews_def['item_id'] = reviews_def['item_id'].astype(int)
    steam_games['id'] = steam_games['id'].astype(int)
    
    # Combinar los DataFrames en función del campo 'item_id'
    merged_data = pd.merge(reviews_def, steam_games, left_on='item_id', right_on='id', how='inner')

    # Filtrar los juegos del año dado
    year_games = merged_data[merged_data['release_date'].dt.year == año]

    # Filtrar los juegos menos recomendados con comentarios negativos
    least_recommended_games = year_games[(year_games['recommend'] == False) & (year_games['sentiment_analysis'] == 0)]

    # Contar el número de veces que cada juego es menos recomendado
    least_recommend_count = least_recommended_games['app_name'].value_counts()

    # Obtener el top 3 de juegos menos recomendados
    menos_recomendados = least_recommend_count.head(3)

    # Damos formato a la respuesta
    respuesta4 = [{"Puesto " + str(i+1): juego} for i, juego in enumerate(menos_recomendados.index)]
    
    return respuesta4

In [69]:
UsersNotRecommend(2015)

[{'Puesto 1': 'call of duty®: black ops iii'},
 {'Puesto 2': 'grand theft auto v'},
 {'Puesto 3': 'batman™: arkham knight'}]

In [62]:
def sentiment_analysis(año: int):
    # Combinamos los dataframes que necesitamos en función del campo 'item_id'
    merged_data = pd.merge(reviews_def, steam_games, left_on='item_id', right_on='id', how='inner')

    # Filtrar las reseñas por el año de lanzamiento
    filtro_resenas = merged_data[merged_data['release_date'].dt.year == año]

    # Contar la cantidad de registros de reseñas categorizados con cada análisis de sentimiento
    sentiment_count = filtro_resenas['sentiment_analysis'].value_counts()

    # Damos formato a la respuesta
    respuesta5 = {
        "Negative": sentiment_count.get(0, 0),
        "Neutral": sentiment_count.get(1, 0),
        "Positive": sentiment_count.get(2, 0)
    }

    return respuesta5

In [63]:
sentiment_analysis(2015)

{'Negative': 1537, 'Neutral': 1409, 'Positive': 3614}

In [72]:
pip freeze

annotated-types==0.6.0
anyio==4.3.0
asttokens==2.4.1
attrs==23.2.0
certifi==2023.7.22
click==8.1.7
colorama==0.4.6
comm==0.1.4
contourpy==1.1.1
cramjam==2.8.3
cycler==0.12.1
debugpy==1.8.0
decorator==5.1.1
distlib==0.3.7
dnspython==2.6.1
email_validator==2.1.1
executing==2.0.1
fastapi==0.111.0
fastapi-cli==0.0.3
fastparquet==2024.2.0
filelock==3.12.4
fonttools==4.43.1
fsspec==2024.3.1
h11==0.14.0
httpcore==1.0.5
httptools==0.6.1
httpx==0.27.0
idna==3.7
ipykernel==6.26.0
ipython==8.17.2
itsdangerous==2.2.0
jedi==0.19.1
Jinja2==3.1.4
joblib==1.4.0
jsonschema==4.22.0
jsonschema-specifications==2023.12.1
jupyter_client==8.5.0
jupyter_core==5.5.0
kiwisolver==1.4.5
markdown-it-py==3.0.0
MarkupSafe==2.1.5
matplotlib==3.8.1
matplotlib-inline==0.1.6
mdurl==0.1.2
nest-asyncio==1.5.8
nltk==3.8.1
numpy==1.26.1
orjson==3.10.3
packaging==23.2
pandas==2.2.2
parso==0.8.3
patsy==0.5.6
Pillow==10.1.0
pipenv==2023.10.3
platformdirs==3.11.0
prompt-toolkit==3.0.39
psutil==5.9.6
pure-eval==0.2.2
pyarrow==16